<a href="https://colab.research.google.com/github/vee234o/promotion-model/blob/main/promotion_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score
import joblib
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OrdinalEncoder
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils.class_weight import compute_sample_weight
from ydata_profiling import ProfileReport
import joblib

/home/codespace/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
#loaded the data
df = pd.read_csv('data/Promotion Dataset (1).csv')
df.head()
df1 = df.copy()

In [21]:

ProfileReport(df).to_file("report.html")

Summarize dataset:  81%|████████  | 21/26 [00:01<00:00, 20.55it/s, Calculate auto correlation]        

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 172.48it/s]


In [22]:
df1.isna().sum()

EmployeeNo                                0
Division                                  0
Qualification                          1679
Gender                                    0
Channel_of_Recruitment                    0
Trainings_Attended                        0
Year_of_birth                             0
Last_performance_score                    0
Year_of_recruitment                       0
Targets_met                               0
Previous_Award                            0
Training_score_average                    0
State_Of_Origin                           0
Foreign_schooled                          0
Marital_Status                            0
Past_Disciplinary_Action                  0
Previous_IntraDepartmental_Movement       0
No_of_previous_employers                  0
Promoted_or_Not                           0
dtype: int64

In [23]:

# dropped missing values in promoted or not column and filled the missing values in the qualification column with its mode
df1 = df1.dropna(subset=['No_of_previous_employers', 'Promoted_or_Not'])
df1['Qualification'] = df1['Qualification'].fillna(df1['Qualification'].mode()[0])


In [24]:
df1.head()

,EmployeeNo,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Promoted_or_Not
0,YAK/S/00001,Commercial Sales and Marketing,MSc MBA and PhD,Female,Direct Internal process,2,1986,12.5,2011,1,0,41,ANAMBRA,No,Married,No,No,0,0
1,YAK/S/00002,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,2,1991,12.5,2015,0,0,52,ANAMBRA,Yes,Married,No,No,0,0
2,YAK/S/00003,Commercial Sales and Marketing,First Degree or HND,Male,Direct Internal process,2,1987,7.5,2012,0,0,42,KATSINA,Yes,Married,No,No,0,0
3,YAK/S/00004,Commercial Sales and Marketing,First Degree or HND,Male,Agency and others,3,1982,2.5,2009,0,0,42,NIGER,Yes,Single,No,No,1,0
4,YAK/S/00006,Information and Strategy,First Degree or HND,Male,Direct Internal process,3,1990,7.5,2012,0,0,77,AKWA IBOM,Yes,Married,No,No,1,0


In [25]:
gender_bias = df.groupby('Gender')['Promoted_or_Not'].mean()
print(gender_bias)


recruit_bias = df.groupby('Channel_of_Recruitment')['Promoted_or_Not'].mean()
print(recruit_bias)

state_bias = df.groupby('State_Of_Origin')['Promoted_or_Not'].mean().sort_values()
print(state_bias.head(3))
print(state_bias.tail(3))

Gender
Female    0.089048
Male      0.082701
Name: Promoted_or_Not, dtype: float64
Channel_of_Recruitment
Agency and others                  0.084280
Direct Internal process            0.083179
Referral and Special candidates    0.121287
Name: Promoted_or_Not, dtype: float64
State_Of_Origin
KWARA     0.067974
ABIA      0.068421
JIGAWA    0.068702
Name: Promoted_or_Not, dtype: float64
State_Of_Origin
TARABA    0.100000
GOMBE     0.103093
ONDO      0.117714
Name: Promoted_or_Not, dtype: float64


In [26]:
bias_data = pd.DataFrame({
    'Category': ['Special Referrals', 'Ondo State', 'Standard Applicants', 'Kwara State'],
    'Promotion_Rate': [12.1, 11.8, 8.4, 6.8]
})

plt.figure(figsize=(10, 6))
bars = plt.bar(bias_data['Category'], bias_data['Promotion_Rate'], color=['firebrick', 'firebrick', 'steelblue', 'steelblue'])

plt.title('Audit Results: Discovered Bias in Historical Data', fontsize=14, pad=20)
plt.ylabel('Promotion Rate (%)', fontsize=12)
plt.xlabel('Group', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.3)

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.2,
             f'{height}%',
             ha='center', va='bottom', fontweight='bold', fontsize=11)

plt.tight_layout()
plt.show()

In [27]:
biased_columns = ['Promoted_or_Not', 'EmployeeNo', 'State_Of_Origin', 'Channel_of_Recruitment']

X = df1.drop(biased_columns, axis=1) 
y = df1['Promoted_or_Not']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y)

In [28]:
#encoded categorical columns
categorical_cols = X_train.select_dtypes(include=['object']).columns
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
#fit and trained them seperately to avoid data leakage
X_train[categorical_cols] = encoder.fit_transform(X_train[categorical_cols])
X_test[categorical_cols] = encoder.transform(X_test[categorical_cols])

In [29]:
#correlated the features and ranked them in descending order
df1.corr(numeric_only=True)['Promoted_or_Not'].sort_values(ascending=False)

Promoted_or_Not             1.000000
Targets_met                 0.224518
Previous_Award              0.201434
Training_score_average      0.178448
Last_performance_score      0.119690
Year_of_birth               0.017991
Year_of_recruitment         0.012287
No_of_previous_employers    0.001690
Trainings_Attended         -0.024345
Name: Promoted_or_Not, dtype: float64

In [30]:
#initialized the model
hgb = HistGradientBoostingClassifier(random_state=42, class_weight = 'balanced', categorical_features='from_dtype')


param_dist = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_iter': [100, 200, 300],        # equivalent to n_estimators
    'max_depth': [7, 9, 12],
    'l2_regularization': [0, 1.0]
}

#used randomized cv to search
random_search = RandomizedSearchCV(
    hgb,
    param_distributions=param_dist,
    n_iter=10,
    cv=3,
    scoring='f1',
    n_jobs=-1,
    random_state=42
)

model = random_search.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [31]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.73      0.84      7015
           1       0.23      0.90      0.37       648

    accuracy                           0.74      7663
   macro avg       0.61      0.81      0.61      7663
weighted avg       0.92      0.74      0.80      7663



In [32]:
ConfusionMatrixDisplay.from_predictions(y_test, y_pred, display_labels=["No", "Yes"], cmap='Blues')
plt.title("Promotion Prediction ")
plt.show()

In [33]:
y_pred = model.predict(X_test)

plt.figure(figsize=(8, 6))
plt.rcParams.update({'font.size': 12})

disp = ConfusionMatrixDisplay.from_predictions(
    y_test, 
    y_pred,
    display_labels=["Not Promoted", "Promoted"],
    cmap=plt.cm.Blues,
    normalize=None
)

plt.title("Confusion Matrix", pad=20)
plt.show()

In [34]:
joblib.dump(model, 'promotion_model.pkl')
joblib.dump(encoder, 'promotion_encoder.pkl')

['promotion_encoder.pkl']

# Project Conclusion and Findings

## 1. Investigation into Bias Allegations
I analyzed historical data to verify staff concerns regarding fairness in the promotion process. My analysis revealed the following:

* **Gender:** I found no evidence of bias against women. The process appears to be neutral regarding gender.
* **Recruitment Channel:** The data confirms a bias here. Staff hired via "Referrals" are promoted significantly more often than standard applicants.
* **State of Origin:** I observed a clear regional gap. Employees from Ondo state are promoted at nearly double the rate of those from Kwara state.

## 2. Proposed Solution: The "Blind Justice" Model
To eliminate these biases, I engineered a **"Blind Justice" Model**.

I deliberately excluded `State_Of_Origin` and `Channel_of_Recruitment` from the training data. By removing these variables, the model is **forced** to calculate promotion probability solely based on meritocratic factors (Performance Score, Targets Met, and Training History).

## 3. Recommendations
1. **Adopt the Blind Justice Model:** Implement this new model as the standard pre-screening tool. Since it is mathematically incapable of seeing "State" or "Connections," it serves as a purely objective judge of performance.
2. **Standardize Reviews:** Management should ensure that "Referral" candidates face the same rigorous performance criteria as standard recruits to close the gap identified in the data.